프롬프트 엔지니어링

# 1. 키워드 추출

## 1-1) 키워드 추출 함수

#### openai API 키 불러오기(환경변수)

In [41]:
import os
import openai

# 환경변수에서 API 키 가져오기
api_key = os.environ.get('OPENAI_API_KEY')

# API 키가 올바르게 설정되었는지 확인 (테스트용)
if not api_key:
    raise ValueError("환경변수 OPENAI_API_KEY가 설정되지 않았습니다.")
#else: 
    #print(f"현재 설정된 API 키: {api_key[:3]}********")  # API 키의 앞 3자리만 출력

# OpenAI 클라이언트 생성
client = openai.OpenAI(api_key=api_key)

In [42]:
def get_keyword(user_input):  
    """OpenAI API를 사용하여 5개의 키워드를 추출하는 함수"""
    try:
        # 시스템 프롬프트 정의
        system_prompt = f"""
        [역할]       
        너는 사람들의 취미 영역을 듣고, 거기서 사람들이 중요하게 생각하는 스마트폰의 기능을 키워드화해서 출력해줘. 
        반드시 아래 기준을 지켜줘야 해. 

        [기준]
        1. 사용자의 취미에 중요한 스마트폰 기능/사양/부품을 추천해줘. 
        2. 5개의 키워드를 쉼표(,)로 구분해서 한 줄로만 제공해줘. 
            네가 출력하는 응답은 키워드 5개 이외의 다른 텍스트를 포함하면 절대 안 돼.
        3. 각 키워드는 한 단어고, 5글자 이내여야 해. 간결할 수록 좋아. 
        4. 키워드는 사람들이 일상에서 많이 사용하는 쉬운 단어를 선택해. 
        5. 너무 기술적이거나 긴 단어는 피해줘(예. '스타일러스', '조리개', '이퀄라이저' 제외).  
        6. 위의 사항들을 꼭 지켜서 알려줘. 

        [예시]
        사용자 입력 : '사진 촬영'
        
        원하는 결과 : '배터리', '사이즈', '화질', '용량', '초점'
        """

        # 사용자 입력 포함
        user_prompt = f'"{user_input}과 관련된 스마트폰 기능 키워드 5개를 알려줘.'

         # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o",  # gpt-4o 또는 gpt-3.5-turbo
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7
        )

        # 응답 데이터 처리
        raw_output = response.choices[0].message.content.strip()
        raw_output = raw_output.replace("'", "").split(',')
        return [res.strip() for res in raw_output]

    except openai.OpenAIError as e: 
        print(f" OpenAI API 오류 발생: {e}")
        return []

    except Exception as e:
        print(f" 예기치 않은 오류 발생: {e}")
        return []


In [43]:
output = get_keyword('등산')
output

['배터리', '방수', '내구성', 'GPS', '무게']

In [40]:
output = get_keyword('운동')
output

['방수', '배터리', 'GPS', '내구성', '무게']

In [4]:
output = get_keyword('만화책 읽기')
output

['화면', '배터리', '용량', '사이즈', '해상도']

In [6]:
import pandas as pd 
user_inputs = ['영상 제작', '사진 촬영', '디지털 아트', '음악 & 사운드', '쇼핑 & 패션', 'SNS & 커뮤니티', '여행 & 탐방', '모바일 게임', '독서 & 필기', '운동 & 건강']

res = []
cnt = 0
while cnt < 3:
    for user_input in user_inputs:
        output = get_keyword(user_input)
        print(f'''
        ▶ user_input
        {user_input}
        
        ▶ output
        {output}
        ''')
        res_output = []
        for i in output:
            res_output.append(i.split(','))
        res.append([user_input, res_output])
    cnt += 1

#pd.DataFrame(res, columns = ['user_input', 'outputs']).sort_values('user_input', ascending = False).to_csv('250221_LLM_4omini_test.csv', encoding = 'utf-8', index = False)


        ▶ user_input
        영상 제작
        
        ▶ output
        ['배터리', '화질', '용량', '렌즈', '화면']
        

        ▶ user_input
        사진 촬영
        
        ▶ output
        ['배터리', '사이즈', '해상도', '렌즈', '소음']
        

        ▶ user_input
        디지털 아트
        
        ▶ output
        ['펜슬', '화질', '화면', '배터리', '용량']
        

        ▶ user_input
        음악 & 사운드
        
        ▶ output
        ['음량', '소리', '배터리', '팬텀', '스트림']
        

        ▶ user_input
        쇼핑 & 패션
        
        ▶ output
        ['화면', '배터리', '카메라', '속도', '용량']
        

        ▶ user_input
        SNS & 커뮤니티
        
        ▶ output
        ['화면', '속도', '앱', '카메라', '배터리']
        

        ▶ user_input
        여행 & 탐방
        
        ▶ output
        ['배터리', '화질', 'GPS', '용량', '사이즈']
        

        ▶ user_input
        모바일 게임
        
        ▶ output
        ['배터리', '화면', '속도', '메모리', '소리']
        

        ▶ user_input
        독서 & 필기
        
        ▶ output
        ['화면', '저장', '배터리',

## 1-2) DB 저장

✅ 1️⃣ 실시간으로 LLM 호출 & DB 저장 후 반환 (권장)  
📌 원리:  

사용자가 웹에서 취미 입력  
DB에서 해당 취미의 키워드 검색  
있으면 바로 반환  
없으면 get_keyword() 실행 후 DB에 저장, 그리고 반환  


아래 내용 확인 필요>

MySQL 테이블 생성 (MySQL에서 실행)

In [ ]:
# 테이블 생성

USE SNS_DB;

CREATE TABLE hobbies (
    id INT AUTO_INCREMENT PRIMARY KEY, # 고유번호
    user_id INT NOT NULL, # 사용자ID
    hobby_area VARCHAR(255) NOT NULL, # 취미 영역
    keyword1 VARCHAR(100), # 키워드 1
    keyword2 VARCHAR(100), # 키워드 2
    keyword3 VARCHAR(100), # 키워드 3
    keyword4 VARCHAR(100), # 키워드 4
    keyword5 VARCHAR(100) # 키워드 5
);

# 샘플 데이터 삽입

INSERT INTO hobbies (user_id, hobby_area, keyword1, keyword2, keyword3, keyword4, keyword5)
VALUES 
(1, '사진촬영', '화질', '색감', '해상도', '밝기', '렌즈'),
(2, '등산', '화질', '배터리', '무게', '방수', 'GPS');

MySQL 서버 접속(python)

In [2]:
host_ip = "15.168.221.131"  
DATABASE = "SNS_DB"
user_id = "lab13"
user_password = "lab13"
mysql_url = f"jdbc:mysql://{host_ip}:3306/{DATABASE}"

In [ ]:
# MySQL에서 취미 데이터를 확인하고 없으면 저장하는 함수
def get_or_store_hobby(user_input):
    """입력된 취미가 MySQL에 존재하는지 확인하고, 없으면 저장 후 반환"""
    try:
        # MySQL 연결
        conn = mysql.connector.connect(
            host='localhost',
            user='lab13',
            password='lab13',
            database='SNS_DB',
            charset='utf8mb4'
        )
        cursor = conn.cursor()

        # 취미가 이미 있는지 확인
        cursor.execute("SELECT keyword1, keyword2, keyword3, keyword4, keyword5 FROM hobbies WHERE hobby_area = %s;", (user_input,))
        result = cursor.fetchone()

        if result:
            # 이미 저장된 취미면, 키워드를 가져와서 반환
            keywords = list(filter(None, result))
            print(f"기존 데이터 활용: {user_input} -> {', '.join(keywords)}")
        else:
            # 취미가 없으면 새로운 키워드 생성 후 DB 저장
            keywords = get_keyword(user_input)
            if keywords:
                insert_query = """
                INSERT INTO hobbies (hobby_area, keyword1, keyword2, keyword3, keyword4, keyword5)
                VALUES (%s, %s, %s, %s, %s, %s);
                """
                cursor.execute(insert_query, (user_input, *keywords))
                conn.commit()
                print(f"새로운 취미 저장: {user_input} -> {', '.join(keywords)}")

        return user_input, keywords

    except mysql.connector.Error as e:
        print(f" MySQL 오류 발생: {e}")
        return user_input, []

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# 2. 스마트폰 추천

### 스마트폰 기종 추천 함수

In [ ]:
import openai
import mysql.connector
from datetime import datetime

def get_recommendation(user_input):
    """MySQL에서 취미 데이터를 가져와 LLM을 사용해 스마트폰을 추천하는 함수"""
    try:
        # DB에서 취미 및 키워드 가져오기 (없으면 생성 후 저장)
        hobby_area, keywords = get_or_store_hobby(user_input)

        if not hobby_area or not keywords:
            print("취미 데이터가 올바르지 않습니다.")
            return "추천할 데이터를 찾을 수 없습니다."


        # 실행일 기준 날짜 가져오기
        current_date = datetime.now().strftime("%Y년 %m월 %d일")
    
        prompt = f"""
        [역할]
        너는 사용자의 취미를 분석하고, 해당 취미에 적합한 스마트폰 기종을 3개 추천하는 AI야.
        사용자의 취미와 중요한 스마트폰 기능을 기반으로 적절한 스마트폰 기종을 추천해줘.
    
        [사용자 정보]
        - 사용자의 취미: {hobby_area}
        - 사용자가 중요하게 생각하는 스마트폰 기능: {', '.join(keywords)}
    
        [추천 기준]
        1. 사용자 취미와 취미생활에 중요한 스마트폰의 기능을 종합적으로 고려해서 추천할 것.
        2. 오늘 날짜 ({current_date})를 기준으로 단종되지 않은 최신 스마트폰 기종 3개를 추천할 것.
        3. 추천할 때 스마트폰 모델명 + 입력된 기능 키워드와 연관된 한 줄 설명을 제공할 것.
    
        [예시]
        1. 아이폰 15 프로 맥스 - 사진 촬영에 최적화된 4800만 화소 카메라와 강력한 이미지 처리 능력
        2. 갤럭시 S23 울트라 - 2억 화소 카메라와 전문가용 촬영 기능을 지원하는 스마트폰
        3. 구글 픽셀 8 프로 - AI 기반 사진 보정 기능이 뛰어난 스마트폰
        """
        
        # GPT-4o 호출
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": prompt}],
            temperature=0.7
        )

        # AI 응답 처리
        smartphone_recommendation = response.choices[0].message.content.strip()
        return smartphone_recommendation

    except openai.error.OpenAIError as e:
        print(f" OpenAI API 오류 발생: {e}")
        return "AI 추천을 불러오는 중 오류가 발생했습니다."

    except mysql.connector.Error as e:
        print(f" MySQL 오류 발생: {e}")
        return "데이터베이스 접근 중 오류가 발생했습니다."

    except Exception as e:
        print(f" 예기치 않은 오류 발생: {e}")
        return "예기치 않은 오류가 발생했습니다."